# Text Classification Using Transformer Networks (BERT)

Some initialization:

In [1]:
!pip3 install datasets
!pip3 install transformers
!pip install -U accelerate
!pip install -U transformers

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

In [3]:
def read_league_data(filename):
    # read csv file
    df = pd.read_csv(filename, header=0)
    # Get only the text and label columns
    return df[["message","toxicity_label"]]

In [4]:
labels = open('classes2.txt').read().splitlines()
data = read_league_data('dataToxic.csv')
print(labels)
data = data.rename(columns={"toxicity_label": "label"})
data['message'] = data['message'].str.replace(',',' ',regex=False)
data['label'] = data['label'].apply(lambda x: 1 if x == 'toxic' else 0)
data

['0', '1']


,message,label
0,report for unskilled player is useless thx <3 ...,0
1,mimimi,0
2,im comming for you riven pfft focus Zed always...,1
3,thx top no flash for what ? he has 2 kill in l...,1
4,IIII ISI K udyr top dnt us see it? CAMP MORE P...,0
...,...,...
88083,gj &gt;&lt; xD i said ss gj stop go alone plz ...,1
88084,i like the new un-do button GET BACK GET BACK ...,1
88085,thx now i can b its ok re top sry thought cait...,0
88086,take t brb swian i was ogin b i was going blue...,1


In [5]:
from sklearn.model_selection import train_test_split

train_df, eval_and_test_df = train_test_split(data, train_size=0.8, random_state
= 4)
eval_df, test_df = train_test_split(eval_and_test_df, train_size=0.5, random_state = 4)
train_df.reset_index(inplace=True, drop=True)
eval_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

print(f'train rows: {len(train_df.index):,}')
print(f'eval rows: {len(eval_df.index):,}')
print(f'test rows: {len(test_df.index):,}')

train rows: 70,470
eval rows: 8,809
test rows: 8,809


In [6]:
eval_df

,message,label
0,i looked at the game too late thoughzed no its...,1
1,can i have next blue? ty amumu uselesss 3 time...,0
2,"-.-"" blizt too report",1
3,help bluw blue heimer noob fizz you dont help ...,1
4,yes lee is in bot no mana,0
...,...,...
8804,... so bad tf tf are u fucking retarded? go mi...,1
8805,nvm can u pls ward or do smth u arnt doing any...,1
8806,ss re i tank GG,0
8807,yes nice ward wow jinx att speed wat ? gj send...,1


In [7]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(train_df)
ds['validation'] = Dataset.from_pandas(eval_df)
ds['test'] = Dataset.from_pandas(test_df)
ds

DatasetDict({
    train: Dataset({
        features: ['message', 'label'],
        num_rows: 70470
    })
    validation: Dataset({
        features: ['message', 'label'],
        num_rows: 8809
    })
    test: Dataset({
        features: ['message', 'label'],
        num_rows: 8809
    })
})

Tokenize the texts:

In [8]:
from transformers import AutoTokenizer

transformer_name = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

In [9]:
def tokenize(examples):
    return tokenizer(examples['message'], truncation=True)

train_ds = ds['train'].map(
    tokenize, 
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
eval_ds = ds['validation'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
train_ds.to_pandas()

Map:   0%|          | 0/70470 [00:00<?, ? examples/s]

Map:   0%|          | 0/8809 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,0,"[101, 26425, 3702, 190, 1120, 1777, 1377, 122,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,0,"[101, 192, 4064, 4097, 28117, 1643, 24102, 149...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,"[101, 179, 4164, 2093, 1169, 1128, 4282, 24181...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"[101, 185, 7223, 17143, 1184, 136, 131, 141, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[101, 174, 1584, 1110, 1139, 1514, 1133, 1119,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
70465,1,"[101, 1302, 1185, 195, 1174, 146, 9424, 1106, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
70466,0,"[101, 3254, 1394, 171, 3329, 1770, 12551, 136,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
70467,0,"[101, 2286, 1105, 10908, 2646, 1499, 1115, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
70468,1,"[101, 8598, 170, 2956, 176, 3361, 1128, 1132, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


Create the transformer model:

In [10]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

# https://github.com/huggingface/transformers/blob/65659a29cf5a079842e61a63d57fa24474288998/src/transformers/models/bert/modeling_bert.py#L1486

class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [11]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    transformer_name,
    #num_labels=len(labels),
    num_labels=2
)

model = (
    BertForSequenceClassification
    .from_pretrained(transformer_name, config=config)
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create the trainer object and train:

In [12]:
from transformers import TrainingArguments

num_epochs = 2
batch_size = 24
weight_decay = 0.01
model_name = f'{transformer_name}-sequence-classification'

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_strategy='epoch',
    weight_decay=weight_decay,
)

In [13]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.163500,0.171548,0.930526
2,0.077700,0.158242,0.951867


TrainOutput(global_step=5874, training_loss=0.16012380981185545, metrics={'train_runtime': 8131.8166, 'train_samples_per_second': 17.332, 'train_steps_per_second': 0.722, 'total_flos': 2.533196138236164e+16, 'train_loss': 0.16012380981185545, 'epoch': 2.0})

Evaluate on the test partition:

In [17]:
test_ds = ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds.to_pandas()

Map:   0%|          | 0/8809 [00:00<?, ? examples/s]

,label,input_ids,token_type_ids,attention_mask
0,1,"[101, 193, 1181, 5576, 3840, 16516, 2050, 2746...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,1,"[101, 28000, 20437, 1377, 1112, 178, 4835, 297...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,0,"[101, 5358, 1204, 189, 21298, 1505, 119, 119, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,0,"[101, 1831, 9925, 1499, 119, 106, 131, 1831, 9...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[101, 11437, 1181, 27453, 1181, 1209, 2222, 89...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
8804,0,"[101, 3146, 102]","[0, 0, 0]","[1, 1, 1]"
8805,0,"[101, 1136, 187, 1193, 1662, 167, 167, 131, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8806,1,"[101, 180, 1468, 1204, 15394, 189, 1182, 3361,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8807,1,"[101, 2592, 24181, 12415, 13280, 1189, 1111, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [18]:
output = trainer.predict(test_ds)
output

PredictionOutput(predictions=array([[-5.082605 ,  4.9232044],
       [-4.681512 ,  4.622045 ],
       [ 5.269686 , -6.060426 ],
       ...,
       [-5.1765056,  5.3667703],
       [-2.4790332,  2.3963077],
       [ 5.188245 , -6.1571345]], dtype=float32), label_ids=array([1, 1, 0, ..., 1, 1, 0]), metrics={'test_loss': 0.13361267745494843, 'test_accuracy': 0.9574299012373708, 'test_runtime': 172.748, 'test_samples_per_second': 50.993, 'test_steps_per_second': 2.13})

In [19]:
from sklearn.metrics import classification_report

y_true = output.label_ids
y_pred = np.argmax(output.predictions, axis=-1)
target_names = ["not toxic", "toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

   not toxic       0.96      0.95      0.95      3821
       toxic       0.96      0.97      0.96      4988

    accuracy                           0.96      8809
   macro avg       0.96      0.96      0.96      8809
weighted avg       0.96      0.96      0.96      8809



In [37]:
labels = open('classes.txt').read().splitlines()
df = pd.read_csv("tagged-data.csv", header=0)
# Get only the text and label columns
df = df[["text","target"]]
print(labels)
df = df.rename(columns={"text": "message"})
df['target'] = df['target'].replace(2,1)
df

['0', '1', '2']


,message,target
0,COMMEND ME TY,0
1,sorry nex,0
2,what is the best soup?,0
3,man that silence on axe,0
4,not coming into play,0
...,...,...
3262,"wt?f?asfU JGOFIDLK,YH",1
3263,you must really suck,1
3264,YOU HAVE IDIOT PLAYER,1
3265,SUPER IDIOT,1


In [38]:
dota_ds = DatasetDict()
dota_ds['test'] = Dataset.from_pandas(df)
dota_ds

DatasetDict({
    test: Dataset({
        features: ['message', 'target'],
        num_rows: 3267
    })
})

In [39]:
test_ds_dota = dota_ds['test'].map(
    tokenize,
    batched=True,
    remove_columns=['message'],
    # remove_columns=['title', 'description', 'text'],
)
test_ds_dota.to_pandas()

Map:   0%|          | 0/3267 [00:00<?, ? examples/s]

,target,input_ids,token_type_ids,attention_mask
0,0,"[101, 18732, 25290, 11680, 2137, 22157, 157, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,0,"[101, 2959, 24928, 1775, 102]","[0, 0, 0, 0, 0]","[1, 1, 1, 1, 1]"
2,0,"[101, 1184, 1110, 1103, 1436, 13128, 136, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"
3,0,"[101, 1299, 1115, 3747, 1113, 16301, 102]","[0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]"
4,0,"[101, 1136, 1909, 1154, 1505, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
...,...,...,...,...
3262,1,"[101, 192, 1204, 136, 175, 136, 1112, 2087, 25...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3263,1,"[101, 1128, 1538, 1541, 13054, 102]","[0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1]"
3264,1,"[101, 19141, 145, 26390, 2036, 10999, 19368, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3265,1,"[101, 156, 18124, 9637, 10999, 19368, 1942, 102]","[0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1]"


In [40]:
output_dota = trainer.predict(test_ds_dota)
output_dota

PredictionOutput(predictions=array([[ 0.5117401 , -0.56011057],
       [ 5.139541  , -6.041044  ],
       [-4.9015102 ,  5.1258955 ],
       ...,
       [-2.0287936 ,  2.0109453 ],
       [ 0.93398744, -1.3075069 ],
       [-3.1485865 ,  2.8868132 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 3.5274, 'test_samples_per_second': 926.188, 'test_steps_per_second': 38.839})

In [41]:
y_true = output_dota.label_ids
y_pred = np.argmax(output_dota.predictions, axis=-1)
target_names = ["not toxic", "toxic"]
print(classification_report(y_true, y_pred, target_names=target_names))

InvalidParameterError: The 'y_true' parameter of classification_report must be an array-like or a sparse matrix. Got None instead.